# SQLAlchemy ORM

Comme précédemment, on aura besoin d'un *engine* pour parler à la base de données :

In [1]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///:memory:", echo=True)

## Créer des tables

Cette fois-ci on ne va pas déclarer les tables explicitements, mais les générer automatiquement à partir des classes Python correspondantes :

In [2]:
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

class Fruit(Base):
    __tablename__ = 'fruits'

    id = Column(Integer, primary_key=True)
    nom = Column(String)
    
    def __repr__(self):
        return f"<Fruit(nom={self.nom})>"

On peut retrouver la définition de la table :

In [3]:
Fruit.__table__

Table('fruits', MetaData(bind=None), Column('id', Integer(), table=<fruits>, primary_key=True, nullable=False), Column('nom', String(), table=<fruits>), schema=None)

Et pour créer les tables en base, comme précédemment :

In [4]:
Base.metadata.create_all(engine)

2019-12-04 23:23:31,226 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-12-04 23:23:31,227 INFO sqlalchemy.engine.base.Engine ()
2019-12-04 23:23:31,228 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-12-04 23:23:31,228 INFO sqlalchemy.engine.base.Engine ()
2019-12-04 23:23:31,229 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("fruits")
2019-12-04 23:23:31,229 INFO sqlalchemy.engine.base.Engine ()
2019-12-04 23:23:31,230 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("fruits")
2019-12-04 23:23:31,230 INFO sqlalchemy.engine.base.Engine ()
2019-12-04 23:23:31,231 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE fruits (
	id INTEGER NOT NULL, 
	nom VARCHAR, 
	PRIMARY KEY (id)
)


2019-12-04 23:23:31,232 INFO sqlalchemy.engine.base.Engine ()
2019-12-04 23:23:31,232 INFO sqlalchemy.engine.base.Engine COMMIT


## L'élément clé : la session

Pour gérer la correspondance entre les lignes des tables dans la base et les objets Python, on va avoir besoin d'un objet qu'on appelle la **session**.

In [5]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)  # étape 1 : créer une « session factory »

db_session = Session()  # étape 2 : instancier une session

## Insérer des données

In [6]:
for nom in ["pomme", "banane", "kiwi"]:
    fruit = Fruit(nom=nom)
    db_session.add(fruit)  # la session enregistre cet objet comme étant nouveau

In [7]:
db_session.commit()  # les nouveau objets sont insérés en base

2019-12-04 23:23:31,253 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-12-04 23:23:31,254 INFO sqlalchemy.engine.base.Engine INSERT INTO fruits (nom) VALUES (?)
2019-12-04 23:23:31,254 INFO sqlalchemy.engine.base.Engine ('pomme',)
2019-12-04 23:23:31,255 INFO sqlalchemy.engine.base.Engine INSERT INTO fruits (nom) VALUES (?)
2019-12-04 23:23:31,256 INFO sqlalchemy.engine.base.Engine ('banane',)
2019-12-04 23:23:31,257 INFO sqlalchemy.engine.base.Engine INSERT INTO fruits (nom) VALUES (?)
2019-12-04 23:23:31,257 INFO sqlalchemy.engine.base.Engine ('kiwi',)
2019-12-04 23:23:31,258 INFO sqlalchemy.engine.base.Engine COMMIT


## Récupérer des données

In [8]:
pomme = db_session.query(Fruit).filter_by(nom="pomme").one()
pomme

2019-12-04 23:23:31,263 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-12-04 23:23:31,264 INFO sqlalchemy.engine.base.Engine SELECT fruits.id AS fruits_id, fruits.nom AS fruits_nom 
FROM fruits 
WHERE fruits.nom = ?
2019-12-04 23:23:31,265 INFO sqlalchemy.engine.base.Engine ('pomme',)


<Fruit(nom=pomme)>

## Modifier des données

In [9]:
pomme.nom = "poire"

In [10]:
db_session.commit()

2019-12-04 23:24:13,242 INFO sqlalchemy.engine.base.Engine UPDATE fruits SET nom=? WHERE fruits.id = ?
2019-12-04 23:24:13,244 INFO sqlalchemy.engine.base.Engine ('poire', 1)
2019-12-04 23:24:13,249 INFO sqlalchemy.engine.base.Engine COMMIT


## Relations

Ajoutons une autre classe, et une relation avec la première :

In [12]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

class Barquette(Base):
    __tablename__ = "barquettes"

    id = Column(Integer, primary_key=True)
    fruit_id = Column(Integer, ForeignKey("fruits.id"), nullable=False)  # clé étrangère
    quantité = Column(Integer, nullable=False)

    fruit = relationship(Fruit)

Base.metadata.create_all(engine)

2019-12-04 23:30:59,096 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("fruits")
2019-12-04 23:30:59,098 INFO sqlalchemy.engine.base.Engine ()
2019-12-04 23:30:59,101 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("barquettes")
2019-12-04 23:30:59,102 INFO sqlalchemy.engine.base.Engine ()
2019-12-04 23:30:59,103 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("barquettes")
2019-12-04 23:30:59,104 INFO sqlalchemy.engine.base.Engine ()
2019-12-04 23:30:59,106 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE barquettes (
	id INTEGER NOT NULL, 
	fruit_id INTEGER NOT NULL, 
	"quantité" INTEGER NOT NULL, 
	PRIMARY KEY (id), 
	FOREIGN KEY(fruit_id) REFERENCES fruits (id)
)


2019-12-04 23:30:59,107 INFO sqlalchemy.engine.base.Engine ()
2019-12-04 23:30:59,108 INFO sqlalchemy.engine.base.Engine COMMIT


In [17]:
kaki = Fruit(nom="kaki")
barquette = Barquette(fruit=kaki, quantité=4)

In [18]:
db_session.add(barquette)

In [19]:
db_session.commit()

2019-12-04 23:33:35,266 INFO sqlalchemy.engine.base.Engine INSERT INTO fruits (nom) VALUES (?)
2019-12-04 23:33:35,268 INFO sqlalchemy.engine.base.Engine ('kaki',)
2019-12-04 23:33:35,274 INFO sqlalchemy.engine.base.Engine INSERT INTO barquettes (fruit_id, "quantité") VALUES (?, ?)
2019-12-04 23:33:35,275 INFO sqlalchemy.engine.base.Engine (4, 4)
2019-12-04 23:33:35,278 INFO sqlalchemy.engine.base.Engine COMMIT


## Références

- https://docs.sqlalchemy.org/en/13/orm/tutorial.html
- https://www.martinfowler.com/eaaCatalog/unitOfWork.html
- https://learning.oreilly.com/library/view/essential-sqlalchemy-2nd/9781491916544/